# Experiment 008: Per-Tree Rotation Optimization

Current fix_direction rotates ALL trees together. Per-tree optimization:
- For each tree, try multiple rotation angles
- Keep the angle that minimizes the bounding box without causing overlaps
- This is a local search that can find better configurations

**Starting point:** 84.712432
**Target:** Improve by 0.5-1 points

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import os
import shutil
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
# ChristmasTree class with rotation support
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._build_polygon()
    
    def _build_polygon(self):
        trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
        base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
        tip_y, tier_1_y, tier_2_y, base_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25'), Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))
        b = self.polygon.bounds
        self.bounds = (b[0]/float(scale_factor), b[1]/float(scale_factor), b[2]/float(scale_factor), b[3]/float(scale_factor))
    
    def set_angle(self, new_angle):
        self.angle = Decimal(str(new_angle))
        self._build_polygon()
    
    def copy(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
# Fast scoring and helper functions
def get_bounds_side(bounds_list):
    if not bounds_list:
        return 0.0
    minx = min(b[0] for b in bounds_list)
    miny = min(b[1] for b in bounds_list)
    maxx = max(b[2] for b in bounds_list)
    maxy = max(b[3] for b in bounds_list)
    return max(maxx - minx, maxy - miny)

def get_side_from_trees(trees):
    return get_bounds_side([t.bounds for t in trees])

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        for idx in tree_index.query(poly):
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def tree_overlaps_others(tree, trees, idx):
    """Check if tree at idx overlaps with any other tree."""
    for j, other in enumerate(trees):
        if j != idx:
            if tree.polygon.intersects(other.polygon) and not tree.polygon.touches(other.polygon):
                return True
    return False

print("Helper functions defined")

Helper functions defined


In [4]:
# Load current best submission
df = pd.read_csv('/home/code/experiments/005_extended_optimization/submission_fd.csv')

def strip_s(val):
    s = str(val)
    return s[1:] if s.startswith('s') else s

dict_of_tree_list = {}
dict_of_side_length = {}

for n in range(1, 201):
    gid = f"{n:03d}"
    group = df[df['id'].str.startswith(f'{gid}_')]
    if len(group) == n:
        trees = []
        for _, row in group.iterrows():
            trees.append(ChristmasTree(
                center_x=strip_s(row['x']),
                center_y=strip_s(row['y']),
                angle=strip_s(row['deg'])
            ))
        dict_of_tree_list[gid] = trees
        dict_of_side_length[gid] = get_side_from_trees(trees)

initial_score = sum(dict_of_side_length[f"{n:03d}"]**2 / n for n in range(1, 201))
print(f"Initial score: {initial_score:.6f}")

Initial score: 84.712432


In [5]:
# Per-tree rotation optimization
def optimize_tree_rotations(trees, angles_to_try=None, verbose=False):
    """Optimize rotation of each tree individually."""
    if angles_to_try is None:
        # Try 72 angles (5-degree increments)
        angles_to_try = np.linspace(0, 355, 72)
    
    n = len(trees)
    best_side = get_side_from_trees(trees)
    improvements = 0
    
    for i in range(n):
        original_angle = float(trees[i].angle)
        best_angle = original_angle
        best_local_side = best_side
        
        for angle in angles_to_try:
            if abs(angle - original_angle) < 0.1:  # Skip current angle
                continue
            
            # Try new angle
            trees[i].set_angle(angle)
            
            # Check for overlaps with this tree
            if not tree_overlaps_others(trees[i], trees, i):
                new_side = get_side_from_trees(trees)
                if new_side < best_local_side - 1e-9:
                    best_local_side = new_side
                    best_angle = angle
        
        # Set to best angle found
        if abs(best_angle - original_angle) > 0.1:
            trees[i].set_angle(best_angle)
            if verbose:
                print(f"  Tree {i}: {original_angle:.1f} -> {best_angle:.1f}, side: {best_side:.6f} -> {best_local_side:.6f}")
            best_side = best_local_side
            improvements += 1
        else:
            trees[i].set_angle(original_angle)
    
    return trees, improvements

print("Per-tree rotation function defined")

Per-tree rotation function defined


In [6]:
# Run per-tree rotation optimization on worst-performing groups
print("\n" + "=" * 60)
print("PER-TREE ROTATION OPTIMIZATION")
print("=" * 60)

# Calculate efficiency for each N
efficiencies = []
for n in range(1, 201):
    gid = f"{n:03d}"
    side = dict_of_side_length[gid]
    tree_area = n * 0.2425  # Each tree has area ~0.2425
    box_area = side ** 2
    efficiency = tree_area / box_area * 100
    efficiencies.append((n, efficiency, side))

# Sort by efficiency (worst first)
efficiencies.sort(key=lambda x: x[1])

print("\nWorst 20 groups by efficiency:")
for n, eff, side in efficiencies[:20]:
    print(f"  N={n:3d}: efficiency={eff:.2f}%, side={side:.6f}")


PER-TREE ROTATION OPTIMIZATION

Worst 20 groups by efficiency:
  N=  1: efficiency=36.67%, side=0.813173
  N= 49: efficiency=49.49%, side=4.900000
  N= 21: efficiency=49.73%, side=3.200000
  N= 50: efficiency=50.50%, side=4.900000
  N= 20: efficiency=50.69%, side=3.093126
  N= 37: efficiency=50.86%, side=4.200000
  N= 53: efficiency=51.41%, side=5.000000
  N= 51: efficiency=51.51%, side=4.900000
  N= 34: efficiency=51.53%, side=4.000000
  N= 19: efficiency=51.73%, side=2.984300
  N= 32: efficiency=51.79%, side=3.870770
  N= 76: efficiency=52.06%, side=5.950000
  N= 22: efficiency=52.10%, side=3.200000
  N= 38: efficiency=52.24%, side=4.200000
  N= 54: efficiency=52.38%, side=5.000000
  N= 52: efficiency=52.52%, side=4.900000
  N= 45: efficiency=52.71%, side=4.550000
  N= 18: efficiency=52.72%, side=2.877563
  N= 77: efficiency=52.74%, side=5.950000
  N= 35: efficiency=53.05%, side=4.000000


In [ ]:
# Optimize the worst groups
print("\n" + "=" * 60)
print("OPTIMIZING WORST GROUPS")
print("=" * 60)

angles = np.linspace(0, 355, 72)  # 5-degree increments
total_improvements = 0

# Focus on worst 50 groups
for n, eff, old_side in efficiencies[:50]:
    gid = f"{n:03d}"
    trees = dict_of_tree_list[gid]
    
    # Make copies to avoid modifying original
    trees_copy = [t.copy() for t in trees]
    
    new_trees, improvements = optimize_tree_rotations(trees_copy, angles, verbose=False)
    new_side = get_side_from_trees(new_trees)
    
    if new_side < old_side - 1e-9:
        # Verify no overlaps
        if not has_overlap(new_trees):
            dict_of_tree_list[gid] = new_trees
            dict_of_side_length[gid] = new_side
            total_improvements += 1
            print(f"N={n:3d}: {old_side:.6f} -> {new_side:.6f} (improved {improvements} trees)")

print(f"\nTotal groups improved: {total_improvements}")

In [ ]:
# Calculate new score
new_score = sum(dict_of_side_length[f"{n:03d}"]**2 / n for n in range(1, 201))
print(f"\nNew score: {new_score:.6f}")
print(f"Improvement: {initial_score - new_score:.6f}")

In [ ]:
# Try fractional translation polish
print("\n" + "=" * 60)
print("FRACTIONAL TRANSLATION POLISH")
print("=" * 60)

def fractional_translation(trees, steps=None, iterations=50):
    """Fine-grained position adjustments."""
    if steps is None:
        steps = [0.01, 0.005, 0.002, 0.001, 0.0005, 0.0002, 0.0001]
    
    directions = [(1,0), (-1,0), (0,1), (0,-1), (1,1), (-1,1), (1,-1), (-1,-1)]
    
    best_side = get_side_from_trees(trees)
    improvements = 0
    
    for _ in range(iterations):
        improved = False
        for i, tree in enumerate(trees):
            original_x = float(tree.center_x)
            original_y = float(tree.center_y)
            
            for step in steps:
                for dx, dy in directions:
                    new_x = original_x + dx * step
                    new_y = original_y + dy * step
                    
                    # Try move
                    tree.center_x = Decimal(str(new_x))
                    tree.center_y = Decimal(str(new_y))
                    tree._build_polygon()
                    
                    if not tree_overlaps_others(tree, trees, i):
                        new_side = get_side_from_trees(trees)
                        if new_side < best_side - 1e-9:
                            best_side = new_side
                            original_x = new_x
                            original_y = new_y
                            improved = True
                            improvements += 1
                            break
                    
                    # Revert
                    tree.center_x = Decimal(str(original_x))
                    tree.center_y = Decimal(str(original_y))
                    tree._build_polygon()
                
                if improved:
                    break
        
        if not improved:
            break
    
    return trees, improvements

print("Fractional translation function defined")

In [ ]:
# Apply fractional translation to worst groups
print("\nApplying fractional translation...")

translation_improvements = 0
for n, eff, old_side in efficiencies[:30]:  # Focus on worst 30
    gid = f"{n:03d}"
    trees = dict_of_tree_list[gid]
    old_side = dict_of_side_length[gid]
    
    new_trees, improvements = fractional_translation(trees, iterations=20)
    new_side = get_side_from_trees(new_trees)
    
    if new_side < old_side - 1e-9:
        if not has_overlap(new_trees):
            dict_of_tree_list[gid] = new_trees
            dict_of_side_length[gid] = new_side
            translation_improvements += 1
            print(f"N={n:3d}: {old_side:.6f} -> {new_side:.6f}")

print(f"\nGroups improved by translation: {translation_improvements}")

In [ ]:
# Calculate final score
final_score = sum(dict_of_side_length[f"{n:03d}"]**2 / n for n in range(1, 201))
print(f"\n=== FINAL RESULTS ===")
print(f"Starting: {initial_score:.6f}")
print(f"After rotation: {new_score:.6f}")
print(f"After translation: {final_score:.6f}")
print(f"Total improvement: {initial_score - final_score:.6f}")
print(f"Target: 68.931058")
print(f"Gap to target: {final_score - 68.931058:.6f}")

In [ ]:
# Save the result
print("\nSaving result...")
rows = []
for n in range(1, 201):
    gid = f"{n:03d}"
    if gid in dict_of_tree_list:
        for i, tree in enumerate(dict_of_tree_list[gid]):
            rows.append({
                'id': f'{gid}_{i}',
                'x': f's{float(tree.center_x)}',
                'y': f's{float(tree.center_y)}',
                'deg': f's{float(tree.angle)}'
            })

result_df = pd.DataFrame(rows)
result_df.to_csv('/home/code/experiments/008_per_tree_rotation/submission_ptr.csv', index=False)
print("Saved to submission_ptr.csv")

In [ ]:
# Validate for overlaps
print("\nValidating...")
overlaps = []
for n in range(1, 201):
    gid = f"{n:03d}"
    if gid in dict_of_tree_list:
        if has_overlap(dict_of_tree_list[gid]):
            overlaps.append(n)

print(f"Overlapping groups: {overlaps}")
print(f"Validation: {'PASSED' if len(overlaps) == 0 else 'FAILED'}")

if len(overlaps) == 0:
    shutil.copy('/home/code/experiments/008_per_tree_rotation/submission_ptr.csv',
                '/home/submission/submission.csv')
    print("Copied to /home/submission/submission.csv")

print(f"\n=== SUMMARY ===")
print(f"Starting: {initial_score:.6f}")
print(f"Final: {final_score:.6f}")
print(f"Improvement: {initial_score - final_score:.6f}")
print(f"Target: 68.931058")
print(f"Gap to target: {final_score - 68.931058:.6f}")